---
### 📦 Información de Versionado con DVC

Este notebook trabaja con datasets versionados usando DVC:

- **Entrada:** `student_performance.csv` (versión limpia después de EDA - `data-v0.2-cleaned`)
- **Salida:** `student_performance_features.csv` (features procesadas listas para modelado)

**Prerrequisitos:**
1. Haber ejecutado el notebook `1.0-el-EDA_cleaning.ipynb`
2. Tener la versión limpia del dataset versionada en DVC

**Para obtener la versión correcta:**
```bash
# Asegurarte de estar en la versión limpia
git checkout data-v0.2-cleaned
dvc checkout

# O simplemente hacer pull de la última versión
dvc pull
```

---


**Exploración y preprocesamiento de datos**

Tarea: Explorar y preprocesar los datos para identificar patrones, tendencias y relaciones significativas.

In [3]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import chi2_contingency
import numpy as np
from scipy.stats import spearmanr
import joblib
from sklearn.decomposition import PCA

In [ ]:
# === Leer dataset limpio versionado con DVC ===
# Este archivo debe ser la versión data-v0.2-cleaned (después de EDA)
DATA_PATH = "../data/processed/student_performance.csv"

print("="*70)
print("📂 Cargando dataset limpio (versión post-EDA)")
print("="*70)
print(f"Ruta: {DATA_PATH}")
print("💡 Asegúrate de tener la versión correcta con: dvc pull")
print("="*70 + "\n")

df_after_eda = pd.read_csv(DATA_PATH)

print(f"✅ Dataset cargado: {df_after_eda.shape[0]} filas, {df_after_eda.shape[1]} columnas")
print("="*70)

In [7]:
df_after_eda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 673 entries, 0 to 672
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Performance           673 non-null    object
 1   Gender                673 non-null    object
 2   Caste                 673 non-null    object
 3   coaching              673 non-null    object
 4   time                  673 non-null    object
 5   Class_ten_education   673 non-null    object
 6   twelve_education      673 non-null    object
 7   medium                673 non-null    object
 8   Class_X_Percentage    673 non-null    object
 9   Class_XII_Percentage  673 non-null    object
 10  Father_occupation     673 non-null    object
 11  Mother_occupation     673 non-null    object
dtypes: object(12)
memory usage: 63.2+ KB


In [9]:
df_after_eda.head()

,Performance,Gender,Caste,coaching,time,Class_ten_education,twelve_education,medium,Class_X_Percentage,Class_XII_Percentage,Father_occupation,Mother_occupation
0,EXCELLENT,MALE,GENERAL,NO,ONE,SEBA,AHSEC,ENGLISH,EXCELLENT,EXCELLENT,DOCTOR,OTHERS
1,EXCELLENT,MALE,OBC,WA,TWO,SEBA,AHSEC,OTHERS,EXCELLENT,EXCELLENT,SCHOOL_TEACHER,HOUSE_WIFE
2,EXCELLENT,MALE,OBC,OA,TWO,OTHERS,CBSE,ENGLISH,EXCELLENT,EXCELLENT,BUSINESS,HOUSE_WIFE
3,EXCELLENT,MALE,GENERAL,WA,ONE,SEBA,AHSEC,OTHERS,EXCELLENT,EXCELLENT,SCHOOL_TEACHER,SCHOOL_TEACHER
4,EXCELLENT,MALE,GENERAL,OA,TWO,SEBA,CBSE,ENGLISH,EXCELLENT,EXCELLENT,COLLEGE_TEACHER,HOUSE_WIFE


In [11]:
for i in df_after_eda.columns:
    print(i)
    print(df_after_eda[i].unique())

Performance
['EXCELLENT' 'VG' 'GOOD' 'AVERAGE']
Gender
['MALE' 'FEMALE' 'MISSING']
Caste
['GENERAL' 'OBC' 'SC' 'ST']
coaching
['NO' 'WA' 'OA']
time
['ONE' 'TWO' 'THREE' 'FOUR' 'FIVE' 'SEVEN']
Class_ten_education
['SEBA' 'OTHERS' 'CBSE']
twelve_education
['AHSEC' 'CBSE' 'OTHERS']
medium
['ENGLISH' 'OTHERS' 'ASSAMESE']
Class_X_Percentage
['EXCELLENT' 'VG' 'GOOD' 'AVERAGE']
Class_XII_Percentage
['EXCELLENT' 'VG' 'GOOD' 'AVERAGE']
Father_occupation
['DOCTOR' 'SCHOOL_TEACHER' 'BUSINESS' 'COLLEGE_TEACHER' 'OTHERS'
 'BANK_OFFICIAL' 'ENGINEER' 'CULTIVATOR']
Mother_occupation
['OTHERS' 'HOUSE_WIFE' 'SCHOOL_TEACHER' 'DOCTOR' 'COLLEGE_TEACHER'
 'BANK_OFFICIAL' 'BUSINESS' 'CULTIVATOR' 'ENGINEER']


Es posible observar que las variables son del siguiente tipo:
Gender: Nominal
Caste: Nominal
Coaching: Nominal
Time: Ordinal
Class_ten_education: Nominal
twelve_education: Nominal
medium: Nominal
Class_X_Percentage: Ordinal
Class_XII_Percentage: Ordinal
Father_occupation: Nominal
Mother_occupation: Nominal

In [13]:
nominal_variables = [
    "Gender",
    "Caste",
    "coaching",
    "time",
    "Class_ten_education",
    "twelve_education",
    "medium",
    "Father_occupation",
    "Mother_occupation"   
]

ordinal_variables = [
    "Class_X_Percentage",
    "Class_XII_Percentage",
]

target_variable = "Performance"

**Ejercicio de frecuencia**

El primer ejercicio que realizamos para entender si había alguna variable que se debiera eliminar fue un ejercicio de frecuencia, para determinar si en alguna de las variables había un valor que fuera demasiado dominante, es decir, que compusiera la gran mayoría de la muestra de esa variable, puesto que eso haría que su impacto en el modelo fuera mínimo. Para este ejercicio el cutoff se definio en 85%, por lo que cualquier variable cuyo valor dominante compusiera más del 85& de la muestra sería descartado.

In [15]:
for i in df_after_eda.columns:
    value_counts = df_after_eda[i].value_counts(normalize=True)
    most_common_freq = value_counts.iloc[0]
    
    print(f"{i}: % Del valor más frecuente: {most_common_freq:.3f}\n")

Performance: % Del valor más frecuente: 0.316

Gender: % Del valor más frecuente: 0.533

Caste: % Del valor más frecuente: 0.499

coaching: % Del valor más frecuente: 0.684

time: % Del valor más frecuente: 0.562

Class_ten_education: % Del valor más frecuente: 0.600

twelve_education: % Del valor más frecuente: 0.557

medium: % Del valor más frecuente: 0.807

Class_X_Percentage: % Del valor más frecuente: 0.768

Class_XII_Percentage: % Del valor más frecuente: 0.600

Father_occupation: % Del valor más frecuente: 0.416

Mother_occupation: % Del valor más frecuente: 0.672



El ejercición de frecuencia no muestra alguna variable que tenga un valor demasiado dominante, la más cercana es Medium, en donde el 80% de la muestra tiene el mismo valor. Por lo tanto, ninguna variable es elminada por frecuencia.

**Chi cuadrada + Cramer's V**

Para las variables nominales, se realizó un ejercicio de chi cuadrada más un ejercicio de Cramer's V para determinar que variables si tenían relación con el Performance, y cuáles no y pueden ser descartadas. Decidimos usar estos ejercicios porque la prueba de chi-cuadrada nos permite determinar si existe una relación estadísticamente significativa entre las variables y el Performance, mientras que el coeficiente de Cramer's V nos ayuda a medir la fuerza la relación, evitando así mantener variables que aunque sean estadísticamente significativas, tengan un efecto muy débil en la práctica.

In [17]:
for i in nominal_variables:
    contingency_table = pd.crosstab(df_after_eda[i], df_after_eda[target_variable])
            
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
            
    print(f"{i} vs {target_variable}:")
    print(f"  Chi-square p-value: {p_value:.6f}")
                
    n = contingency_table.sum().sum()
    cramers_v = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))
    print(f"  Cramer's V (effect size): {cramers_v:.3f}\n")

Gender vs Performance:
  Chi-square p-value: 0.193276
  Cramer's V (effect size): 0.080

Caste vs Performance:
  Chi-square p-value: 0.000000
  Cramer's V (effect size): 0.468

coaching vs Performance:
  Chi-square p-value: 0.000118
  Cramer's V (effect size): 0.143

time vs Performance:
  Chi-square p-value: 0.209195
  Cramer's V (effect size): 0.097

Class_ten_education vs Performance:
  Chi-square p-value: 0.053226
  Cramer's V (effect size): 0.096

twelve_education vs Performance:
  Chi-square p-value: 0.265834
  Cramer's V (effect size): 0.075

medium vs Performance:
  Chi-square p-value: 0.000212
  Cramer's V (effect size): 0.139

Father_occupation vs Performance:
  Chi-square p-value: 0.000001
  Cramer's V (effect size): 0.182

Mother_occupation vs Performance:
  Chi-square p-value: 0.000661
  Cramer's V (effect size): 0.161



Tras realizar el ejercicio, encontramos que las variables "Gender", "time", "Class_ten_education", "twelve_education" no tienen un efecto significativo en el Performance, por lo que pueden ser descartados del modelo.

In [19]:
df_post_nominal = df_after_eda.drop(["Gender", "time", "Class_ten_education", "twelve_education"], axis = 1)
nominal_variables = [
    "Caste",
    "coaching",
    "medium",
    "Father_occupation",
    "Mother_occupation"   
]

**Correlación de Spearman**

El siguiente ejercicio que se realizó fue la correlación de Spearman, en donde codificamos las variables ordinales para encontrar si tenía algún efecto con el Performance o si podían ser descartados del modelo.

In [21]:
ordinal_mapping = {
    "Class_X_Percentage": {
        'EXCELLENT': 3,
        'VG': 2,
        'GOOD': 1,
        'AVERAGE': 0
    },
    "Class_XII_Percentage": {
        'EXCELLENT': 3,
        'VG': 2,
        'GOOD': 1,
        'AVERAGE': 0
    }
    
}

target_mapping = {
    "Performance":{
        'EXCELLENT': 3,
        'VG': 2,
        'GOOD': 1,
        'AVERAGE': 0
    }
}

In [23]:
df_post_nominal['Class_X_Percentage_encoded'] = df_post_nominal['Class_X_Percentage'].map(ordinal_mapping['Class_X_Percentage'])
df_post_nominal['Class_XII_Percentage_encoded'] = df_post_nominal['Class_XII_Percentage'].map(ordinal_mapping['Class_XII_Percentage'])

df_post_nominal['Performance_encoded'] = df_post_nominal['Performance'].map(target_mapping['Performance'])

In [25]:
print("Class_X_Percentage vs Performance")
correlation1, p_value1 = spearmanr(df_post_nominal['Class_X_Percentage_encoded'], 
                                  df_post_nominal['Performance_encoded'],
                                  nan_policy='omit')
print(f"Spearman correlation: {correlation1:.3f}")
print(f"p-value: {p_value1:.6f}\n")

# Analyze Class_XII_Percentage vs Performance  
print("Class_XII_Percentage vs Performance")
correlation2, p_value2 = spearmanr(df_post_nominal['Class_XII_Percentage_encoded'], 
                                  df_post_nominal['Performance_encoded'],
                                  nan_policy='omit')
print(f"Spearman correlation: {correlation2:.3f}")
print(f"p-value: {p_value2:.6f}")

Class_X_Percentage vs Performance
Spearman correlation: 0.209
p-value: 0.000000

Class_XII_Percentage vs Performance
Spearman correlation: 0.274
p-value: 0.000000


El ejercicio nos muestra que ambas variables tienen un efecto en el modelo, por lo que no son descartadas.

**Códificación de variables nominales**

Se utiliza onhot encoding para las variables nominales, las variables ordinales ya habían sido códificadas por lo que solo se elimina las variables originales sin códificar.

In [27]:
def onehot_encode_nominal_variables(df, nominal_variables, drop_original=True):

    onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')
    
    onehot_encoded = onehot_encoder.fit_transform(df[nominal_variables])
    
    onehot_feature_names = onehot_encoder.get_feature_names_out(nominal_variables)
    
    onehot_df = pd.DataFrame(onehot_encoded, columns=onehot_feature_names, index=df.index)
    
    if drop_original:
        df_result = df.drop(columns=nominal_variables)
    else:
        df_result = df.copy()
    
    df_result = pd.concat([df_result, onehot_df], axis=1)
    
    return df_result, onehot_encoder

df_encoded, encoder = onehot_encode_nominal_variables(df_post_nominal, nominal_variables, drop_original = True)

In [31]:
joblib.dump(encoder, 'onehot_encoder.pkl')

['onehot_encoder.pkl']

In [33]:
df_encoded_final = df_encoded.drop(["Performance", "Class_X_Percentage", "Class_XII_Percentage"], axis = 1)

In [35]:
df_encoded_final

,Class_X_Percentage_encoded,Class_XII_Percentage_encoded,Performance_encoded,Caste_OBC,Caste_SC,Caste_ST,coaching_OA,coaching_WA,medium_ENGLISH,medium_OTHERS,...,Father_occupation_OTHERS,Father_occupation_SCHOOL_TEACHER,Mother_occupation_BUSINESS,Mother_occupation_COLLEGE_TEACHER,Mother_occupation_CULTIVATOR,Mother_occupation_DOCTOR,Mother_occupation_ENGINEER,Mother_occupation_HOUSE_WIFE,Mother_occupation_OTHERS,Mother_occupation_SCHOOL_TEACHER
0,3,3,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3,3,3,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,3,3,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,3,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,3,3,3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,3,3,0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
669,2,2,0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
670,2,1,0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
671,3,3,2,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


**PCA**

Se realiza un ejercicio de PCA para reducir la dimensionalidad de las características con las que contamos, mientras se mantenga un 95% de varianza.

In [41]:
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(df_encoded_final.drop(columns=['Performance_encoded']))

In [49]:
X_pca.shape

(673, 15)

In [55]:
X_pca.ndim

2

In [65]:
print(f"Varianza total explicada: {pca.explained_variance_ratio_.sum():.3f}")

Varianza total explicada: 0.957


In [73]:
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
df_pca = pd.DataFrame(X_pca, columns=pca_columns, index=df_encoded.index)

df_pca['Performance'] = df_encoded_final['Performance_encoded'].values

df_pca.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,Performance
0,-0.803234,0.597979,0.285577,0.770143,-0.017667,0.023372,-0.063243,0.451023,0.984059,0.048342,-0.116927,0.258324,-0.415056,-0.299239,-0.045746,3
1,-0.383741,0.558693,-0.664729,-1.081237,1.002544,-0.000102,0.260944,-0.398450,0.258088,-0.143168,0.352092,0.023940,0.116888,0.069941,-0.081184,3
2,-0.665234,-0.025918,-0.910224,0.979597,0.610582,0.515577,0.055653,-0.029394,-0.367403,-0.276059,0.247405,0.025235,0.048601,0.496631,-0.060994,3
3,-0.507289,1.468973,0.208351,-0.967490,0.318739,-0.599155,0.271468,-0.029258,-0.281890,-0.145417,0.155879,-0.050884,-0.074240,0.066675,-0.163159,3
4,-0.676385,-0.010505,-0.672490,0.781445,-0.002355,-0.421630,0.204497,-0.090873,0.031931,-0.006785,-0.248873,-0.298813,-0.188266,0.556631,0.750617,3


**Guardar Dataset de Features y Versionar con DVC**

In [ ]:
# === Guardar dataset de features procesadas ===
OUTPUT_PATH = "../data/processed"
OUTPUT_FILENAME = "student_performance_features.csv"
OUTPUT_FULL_PATH = f"{OUTPUT_PATH}/{OUTPUT_FILENAME}"

df_pca.to_csv(OUTPUT_FULL_PATH, index=False)

print("="*70)
print("✅ Dataset de features procesadas guardado exitosamente")
print("="*70)
print(f"📂 Ruta: {OUTPUT_FULL_PATH}")
print(f"📊 Shape: {df_pca.shape}")
print(f"📋 Columnas: {list(df_pca.columns)}")
print("\n" + "="*70)
print("📦 SIGUIENTE PASO: Versionar con DVC")
print("="*70)
print("\nEjecuta los siguientes comandos en la terminal:\n")
print("# 1. Agregar el archivo a DVC")
print(f"dvc add {OUTPUT_FULL_PATH}")
print("\n# 2. Commitear el cambio a Git")
print(f"git add {OUTPUT_FULL_PATH}.dvc")
print('git commit -m "feat: add engineered features with PCA and encoding"')
print("\n# 3. Crear tag de versión")
print('git tag -a "data-v0.3-features" -m "Version 0.3: Features ready for modeling"')
print("\n# 4. Subir los datos al remote de DVC")
print("dvc push")
print("\n# 5. Subir el commit a Git")
print("git push && git push --tags")
print("\n" + "="*70)
print("💡 Este dataset está listo para ser usado en el entrenamiento de modelos")
print("="*70)